# 영화 리뷰 감정 분석
**Recurrent Neural Network (RNN) 을 이용해 IMDB 데이터를 가지고 텍스트 감정분석을 해 봅시다.**

```
'먹기위해 산다.'
'살기위해 먹는다.'
```

우리가 일상에서 읽고 쓰는 문서, 혹은 언어는 대표적인 순차적 데이터라고 할 수 있습니다. 위의 두 문장에서 볼 수 있듯, 언어는 단어나 문장의 순서에 큰 영향을 받으며, 이러한 언어의 특성덕에 RNN은 '자연어 처리'(Natural Language Processing) 분야에 매우 큰 부각을 나타내 왔습니다. 이번 프로젝트를 통해 자연어 처리 분야 중 가장 기본적이라고 할 수 있는 '텍스트 감정분석'(Sentiment Analysis) 을 같이 구현하고 공부해 보겠습니다.

당연한 이야기겠지만, 자연어 처리를 위해선 텍스트나 언어 형태의 데이터가 필요합니다. 우리가 사용할 IMDB 데이터셋은 25,000건의 영화 리뷰를 가지고 있습니다. 각 리뷰는 다수의 영어 문장들로 이루어져 있으며, 긍정적인 영화 리뷰는 2로, 부정적인 영화 리뷰는 1로 레이블링 되어 있습니다. 영화 리뷰 데이터를 RNN 에 입력시켜 리뷰의 전체 내용을 이해하여 압축하고, 이렇게 압축된 리뷰를 기반으로 영화 리뷰가 긍정적인지 부정적인지 판단해주는 간단한 분류 모델을 구현하는 것이 이번 프로젝트의 목표입니다.

가장 먼저 모델 구현과 학습에 필요한 라이브러리 들을 임포트 해 줍니다. 여기서 처음 등장하는 torchtext 는 자연어 처리 딥러닝에 쓰이는 다양한 데이터셋들을 포함하고 있으며, 데이터셋을 학습에 편한 형태로 바꿔주는 기능을 해 주는 편리한 오픈소스입니다.

In [ ]:
import os
import sys
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets

모델 구현과 학습에 필요한 하이퍼파라미터 들을 정의해 줍니다.

In [2]:
# get hyper parameters
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 40
torch.manual_seed(42)
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

## RNN 기반 모델 클래스 생성하기

BasicLSTM 라고 하는 RNN 을 포함하는 신경망 모델을 만들어 보겠습니다. 여타 다른 신경망 모델과 같이 파이토치의 nn.Module 을 상속받습니다.

```python
class BasicLSTM(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicLSTM, self).__init__()
        print("Building Basic LSTM model...")
```

### RNN 에 필요한 패러미터 정의하기

IMDB 와 같은 자연어 데이터에선 모든 단어들이 랭크 1의 텐서로 정의됩니다.
이 때 각 단어를 나타내는 텐서 속 특성값의 수가 바로 embed 라는 변수입니다.
영화평 속의 모든 단어들이 텐서로 나타내어 지다면, 영화평 전체는 곧 텐서의 배열이라고 할 수 있겠지요.

```python
class BasicLSTM(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicLSTM, self).__init__()
        print("Building Basic LSTM model...")
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
```



In [3]:
# class BasicRNN(nn.Module):
#     """
#         Basic RNN
#     """
#     def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
#         super(BasicRNN, self).__init__()
#         print("Building Basic RNN model...")
#         self.n_layers = n_layers
#         self.hidden_dim = hidden_dim

#         self.embed = nn.Embedding(n_vocab, embed_dim)
#         self.dropout = nn.Dropout(dropout_p)
#         self.rnn = nn.RNN(embed_dim, hidden_dim, n_layers,
#                           dropout=dropout_p, batch_first=True)
#         self.out = nn.Linear(self.hidden_dim, n_classes)

#     def forward(self, x):
#         embedded = self.embed(x)  #  [b, i] -> [b, i, e]
#         _, hidden = self.rnn(embedded)
#         self.dropout(hidden)
#         hidden = hidden.squeeze()
#         logit = self.out(hidden)  # [b, h] -> [b, o]
#         return logit

class BasicLSTM(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicLSTM, self).__init__()
        print("Building Basic LSTM model...")
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(embed_dim, self.hidden_dim,
                            num_layers=self.n_layers,
                            dropout=dropout_p,
                            batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)  #  [b, i] -> [b, i, e]
        h_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.lstm(x, h_0)  # [i, b, h]
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.out(h_t)  # [b, h] -> [b, o]
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return (
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        )

In [4]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # index align
        optimizer.zero_grad()
        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()
        if b % 100 == 0:
            corrects = (logit.max(1)[1].view(y.size()).data == y.data).sum()
            accuracy = 100.0 * corrects / batch.batch_size
            sys.stdout.write(
                '\rBatch[%d] - loss: %.6f  acc: %.2f' %
                (b, loss.item(), accuracy))

In [5]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, avg_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # index align
        logit = model(x)
        loss = F.cross_entropy(logit, y, size_average=False)
        avg_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = avg_loss / size
    accuracy = 100.0 * corrects / size
    return avg_loss, accuracy

# IMDB 데이터셋 가져오기

In [6]:
# load data
print("\nLoading data...")
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
TEXT.build_vocab(train_data, min_freq=5)
LABEL.build_vocab(train_data)

train_iter, test_iter = data.BucketIterator.splits(
        (train_data, test_data), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

vocab_size = len(TEXT.vocab)
n_classes = len(LABEL.vocab) - 1


Loading data...


In [7]:
print("[TRAIN]: %d \t [TEST]: %d \t [VOCAB] %d \t [CLASSES] %d"
      % (len(train_iter),len(test_iter), vocab_size, n_classes))

[TRAIN]: 391 	 [TEST]: 391 	 [VOCAB] 46159 	 [CLASSES] 2


In [ ]:
model = BasicLSTM(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

print(model)

Building Basic LSTM model...
BasicLSTM(
  (embed): Embedding(46159, 128)
  (dropout): Dropout(p=0.5)
  (lstm): LSTM(128, 256, batch_first=True, dropout=0.5)
  (out): Linear(in_features=256, out_features=2, bias=True)
)


/home/keon/.local/lib/python3.5/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, test_iter)

    print("\n[Epoch: %d] val_loss:%5.2f | acc:%5.2f" % (e, val_loss, val_accuracy))
    
    # Save the model if the validation loss is the best we've seen so far.
#     if not best_val_loss or val_loss < best_val_loss:
#         if not os.path.isdir("snapshot"):
#             os.makedirs("snapshot")
#         torch.save(model.state_dict(), './snapshot/convcnn.pt')
#         best_val_loss = val_loss